In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re
import gc
import matplotlib.pyplot as plt
import pyarrow.feather as feather
import seaborn as sns
import datatable as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix


from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc

import xgboost as xgb
from  xgboost import XGBRFClassifier

In [3]:
train = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")
tan_lab =pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train = train.merge(tan_lab, how = 'inner', on = 'customer_ID')

In [4]:
lab = LabelEncoder()
train['customer_ID']= lab.fit_transform(train['customer_ID'])

In [5]:
train = train.groupby('customer_ID').tail(1).set_index('customer_ID')

In [6]:
test = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/test.parquet")

In [7]:
test['customer_ID']= lab.fit_transform(test['customer_ID'])

In [8]:
test = test.groupby('customer_ID').tail(1).set_index('customer_ID')

In [9]:
y = train.target
X = train.drop(["target","S_2"],axis=1)


test = test.drop(["S_2"],axis=1)

In [10]:
X = X.fillna(-123)
test = test.fillna(-123)

In [11]:
cat_cols =['B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']
num_cols = [col for col in X.columns if col not in cat_cols ]


In [12]:
D_n_cols = [col for col in num_cols if col.startswith("D")]
S_n_cols = [col for col in num_cols if col.startswith("S")]
P_n_cols = [col for col in num_cols if col.startswith("P")]
B_n_cols = [col for col in num_cols if col.startswith("B")]
R_n_cols = [col for col in num_cols if col.startswith("R")]
D_c_cols = [col for col in cat_cols if col.startswith("D")]
B_c_cols = [col for col in cat_cols if col.startswith("B")] 

In [13]:
X["D_mean"]=X[D_n_cols].mean(axis=1)
X["S_mean"]=X[S_n_cols].mean(axis=1)
X["P_mean"]=X[P_n_cols].mean(axis=1)
X["B_mean"]=X[B_n_cols].mean(axis=1)
X["R_mean"]=X[R_n_cols].mean(axis=1)

X["D_max"]=X[D_n_cols].max(axis=1)
X["S_max"]=X[S_n_cols].max(axis=1)
X["P_max"]=X[P_n_cols].max(axis=1)
X["B_max"]=X[B_n_cols].max(axis=1)
X["R_max"]=X[R_n_cols].max(axis=1)

X["D_min"]=X[D_n_cols].min(axis=1)
X["S_min"]=X[S_n_cols].min(axis=1)
X["P_min"]=X[P_n_cols].min(axis=1)
X["B_min"]=X[B_n_cols].min(axis=1)
X["R_min"]=X[R_n_cols].min(axis=1)

X["D_c_unique"]=X[D_c_cols].nunique(axis=1)
X["B_c_unique"]=X[B_c_cols].nunique(axis=1)

X['mat_num'] =X["B_mean"]+ X["D_mean"] +X["S_mean"] +X["P_mean"] + X["R_mean"]

test["D_mean"]=test[D_n_cols].mean(axis=1)
test["S_mean"]=test[S_n_cols].mean(axis=1)
test["P_mean"]=test[P_n_cols].mean(axis=1)
test["B_mean"]=test[B_n_cols].mean(axis=1)
test["R_mean"]=test[R_n_cols].mean(axis=1)

test["D_max"]=test[D_n_cols].max(axis=1)
test["S_max"]=test[S_n_cols].max(axis=1)
test["P_max"]=test[P_n_cols].max(axis=1)
test["B_max"]=test[B_n_cols].max(axis=1)
test["R_max"]=test[R_n_cols].max(axis=1)

test["D_min"]=test[D_n_cols].min(axis=1)
test["S_min"]=test[S_n_cols].min(axis=1)
test["P_min"]=test[P_n_cols].min(axis=1)
test["B_min"]=test[B_n_cols].min(axis=1)
test["R_min"]=test[R_n_cols].min(axis=1)

test["D_c_unique"]=test[D_c_cols].nunique(axis=1)
test["B_c_unique"]=test[B_c_cols].nunique(axis=1)

test['mat_num'] =test["B_mean"]+ test["D_mean"] +test["S_mean"] +test["P_mean"] + test["R_mean"]


print(X.shape,test.shape)

(458913, 206) (924621, 206)


In [14]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

X[cat_cols]= ordinal_encoder.fit_transform(X[cat_cols])
test[cat_cols] = ordinal_encoder.transform(test[cat_cols])

X.shape,test.shape

((458913, 206), (924621, 206))

In [15]:
scaler = MinMaxScaler()

X[num_cols] = scaler.fit_transform(X[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

X[cat_cols] = scaler.fit_transform(X[cat_cols])
test[cat_cols] = scaler.transform(test[cat_cols])

In [16]:
smote = SMOTE(sampling_strategy='minority')
X_sm,y_sm = smote.fit_resample(X,y)
y_sm.value_counts()

0    340085
1    340085
Name: target, dtype: int64

In [17]:
X_f = np.array(X_sm)
y_f = np.array(y_sm)

test = np.array(test)

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(X_f, y_f, test_size = 0.20, random_state=2021,stratify=y_f)
print("x_train shape",X_train.shape)
print("x_test shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_test shape",Y_val.shape)

x_train shape (544136, 206)
x_test shape (136034, 206)
y_train shape (544136,)
y_test shape (136034,)


In [19]:
param={'booster': 'dart',
 'lambda': 0.000156469894565185,
 'alpha': 0.10410870792558755,
 'subsample': 0.6674526274907548,
 'colsample_bytree': 0.5480184568542634,
 'max_depth': 9,
 'min_child_weight': 8,
 'eta': 0.16148473311025344,
 'gamma': 3.3872652561508726e-05,
 'grow_policy': 'lossguide',
 'sample_type': 'weighted',
 'normalize_type': 'forest',
 'rate_drop': 2.519962453550491e-07,
 'skip_drop': 0.0037991281511123055}

In [20]:
gbm =LGBMClassifier(**param).fit(X_train, Y_train, 
                                       eval_set=[(X_train, Y_train), (X_val, Y_val)],
                                       callbacks=[early_stopping(200), log_evaluation(500)],
                                       eval_metric=['auc','binary_logloss'])

[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] lambda_l2 is set with lambda=0.000156469894565185, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.000156469894565185
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Unknown parameter: normalize_type
[LightGBM] [Warning] Unknown parameter: grow_policy
[LightGBM] [Warning] Unknown parameter: sample_type
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.16148473311025344 will be ignored. Current value: learning_rate=0.1
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.985604	training's binary_logloss: 0.154104	valid_1's auc: 0.985008	valid_1's binary_logloss: 0.157068


In [21]:
predictions = gbm.predict(X_val)

In [22]:
cf = classification_report(Y_val,predictions)
print(cf)

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     68017
           1       0.93      0.93      0.93     68017

    accuracy                           0.93    136034
   macro avg       0.93      0.93      0.93    136034
weighted avg       0.93      0.93      0.93    136034



In [23]:
prd_test = gbm.predict_proba(test)

In [24]:
preds = pd.DataFrame(prd_test)
pred_final = np.array(preds[1])
pred_final

array([0.04753891, 0.00248608, 0.04272379, ..., 0.53509831, 0.52171269,
       0.09429967])

In [25]:
submission = pd.read_csv("../input/amex-default-prediction/sample_submission.csv")

In [26]:
submission['prediction']=pred_final

In [27]:
submission.to_csv("submission.csv",index=False)